<a href="https://colab.research.google.com/github/js-lee-AI/Abnormal-detection/blob/master/research/ensemble_NLP/NLP_soft_voting_(MNLI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece

     |████████████████████████████████| 1.6MB 8.1MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 1.2 MB 8.2 MB/s 


In [ ]:
!pip install transformers	
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

     |████████████████████████████████| 2.5 MB 8.1 MB/s 
     |████████████████████████████████| 3.3 MB 53.0 MB/s 
     |████████████████████████████████| 895 kB 68.7 MB/s 
ERROR: torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl is not a supported wheel on this platform.


In [ ]:
from transformers import pipeline
import torch
import torch.nn as nn
import torchvision.datasets as dsets

In [ ]:
torch.cuda.is_available() 


True

In [ ]:
cuda = torch.device('cuda')
cuda

device(type='cuda')

# roberta-large-mnli

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nli_model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli').cuda()
tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
# 0. contradiction
# 1. neutral
# 2. entailment

# label = ['travel', 'exploration', 'dancing', 'cooking']

premise = 'one day I will see the world'
label = 'travel'
hypothesis = f'This example is {label}.'
# hypothesis = 'travel'

# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
logits_roberta = nli_model(x.to(device))[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits_roberta[:,[2,0]]
probs_roberta = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs_roberta[:,1]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_lengt

In [ ]:
probs_roberta

tensor([[0.9924, 0.0076]], device='cuda:0', grad_fn=<SoftmaxBackward>)

#bart-large-mnli

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').cuda()
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
# 0. contradiction
# 1. neutral
# 2. entailment

# label = ['travel', 'exploration', 'dancing', 'cooking']
premise = 'one day I will see the world'
label = 'travel'
hypothesis = f'This example is {label}.'


# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
logits_roberta = nli_model(x.to(device))[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits_roberta[:,[1,0]]
probs_bart = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs_bart[:,1]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


In [ ]:
probs_bart

tensor([[0.9844, 0.0156]], device='cuda:0', grad_fn=<SoftmaxBackward>)

# Distilbart-mnli-12-9

In [ ]:
# pose sequence as a NLI premise and label as a hypothesis
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nli_model = AutoModelForSequenceClassification.from_pretrained('valhalla/distilbart-mnli-12-9').cuda()
tokenizer = AutoTokenizer.from_pretrained('valhalla/distilbart-mnli-12-9')
# 0. contradiction
# 1. neutral
# 2. entailment

# label = ['travel', 'exploration', 'dancing', 'cooking']
premise = 'one day I will see the world'
label = 'travel'
hypothesis = f'This example is {label}.'


# run through model pre-trained on MNLI
x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
logits_roberta = nli_model(x.to(device))[0]

# we throw away "neutral" (dim 1) and take the probability of
# "entailment" (2) as the probability of the label being true 
entail_contradiction_logits = logits_roberta[:,[2,0]]
probs_distilbart = entail_contradiction_logits.softmax(dim=1)
prob_label_is_true = probs_distilbart[:,1]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


In [ ]:
probs_distilbart

tensor([[0.9976, 0.0024]], device='cuda:0', grad_fn=<SoftmaxBackward>)

# Soft voting

In [ ]:
total_probs = torch.div(probs_roberta[0] + probs_bart[0] + probs_distilbart[0], 3)
total_probs

tensor([0.9915, 0.0085], device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
result = torch.argmax(total_probs).tolist()
if result==0: print('entailment') 
elif result==1: print('contradiction')

entailment


# Dataset download (MNLI)

In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 262 kB 9.5 MB/s 
     |████████████████████████████████| 243 kB 57.1 MB/s 
     |████████████████████████████████| 118 kB 67.6 MB/s 


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece

In [ ]:
from datasets import load_dataset, load_metric
actual_task = 'mnli'
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [ ]:
dataset
# labels
# 0 : entailment
# 1 : neutral
# 2 : contradiction

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
dataset['train']

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 392702
})

In [ ]:
dataset['test_matched']

Dataset({
    features: ['premise', 'hypothesis', 'label', 'idx'],
    num_rows: 9796
})

In [ ]:
dataset['train'][0]

{'hypothesis': 'Product and geography are what make cream skimming work. ',
 'idx': 0,
 'label': 1,
 'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.'}

In [ ]:
dataset['train'][0:10]

{'hypothesis': ['Product and geography are what make cream skimming work. ',
  'You lose the things to the following level if the people recall.',
  'A member of my team will execute your orders with immense precision.',
  'This information belongs to them.',
  'The tennis shoes have a range of prices.',
  "I'm upset that my walkman broke and now I have to turn the stereo up really loud.",
  'Most of the Christian mosaics were destroyed by Muslims.  ',
  "Slate had an opinion on Jackson's findings.",
  'Heterosexuals.',
  'Place des Vosges is constructed entirely of gray marble.'],
 'idx': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'label': [1, 0, 0, 0, 1, 0, 1, 0, 2, 2],
 'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.',
  'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replac

In [ ]:
roberta_nli_model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli').cuda()
roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
roberta_model = [roberta_nli_model, roberta_tokenizer]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def evaluate_roberta(model, tokenizer_, dset, dlen=100) :
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  nli_model =  model
  tokenizer = tokenizer_
  # 0. contradiction   ent  neu
  # 1. neutral  con  ent
  # 2. entailment  neu  con
  true_cnt = 0
  false_cnt = 0

  for data in dset:
    premise = data['premise']
    hypothesis = data['hypothesis']
  
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
    logits_roberta = nli_model(x.to(device))[0]


    entail_contradiction_logits = logits_roberta[:,[2,1,0]]
    probs_roberta = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs_roberta[:,1]
    result = torch.argmax(probs_roberta).tolist()
    if result== data['label'] : true_cnt+=1
    else : false_cnt+=1

    # 빨리 종료 (데이터 dlen개에 대해서만 accuracy 측정)
    if data['idx'] == dlen : break
      
  print('Roberta - accuracy : {:.2f}%'.format(100*true_cnt/(true_cnt+false_cnt)))
  return probs_roberta

In [ ]:
evaluate_roberta(roberta_nli_model, roberta_tokenizer, dataset['train'])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


Roberta - accuracy : 97.03%


tensor([[1.8094e-04, 3.2623e-04, 9.9949e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

In [ ]:
bart_nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').cuda()
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
bart_model = [bart_nli_model, bart_tokenizer]

In [ ]:
def evaluate_bart(model, tokenizer_, dset, dlen=100) :
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  nli_model = model
  tokenizer = tokenizer_
  # 0. contradiction   ent  neu
  # 1. neutral  con  ent
  # 2. entailment  neu  con
  true_cnt = 0
  false_cnt = 0

  for data in dset:
    premise = data['premise']
    hypothesis = data['hypothesis']
  
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
    logits_bart = nli_model(x.to(device))[0]

    entail_contradiction_logits = logits_bart[:,[2,1,0]]
    probs_bart = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs_bart[:,1]
    result = torch.argmax(probs_bart)
    if result == data['label'] : true_cnt+=1
    else : false_cnt+=1

    # 빨리 종료
    if data['idx'] == dlen : break
  print('bart - accuracy : {:.2f}%'.format(100*true_cnt/(true_cnt+false_cnt)))
  return probs_bart

In [ ]:
evaluate_bart(bart_nli_model, bart_tokenizer, dataset['train'])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


bart - accuracy : 97.03%


tensor([[8.4872e-05, 1.6819e-04, 9.9975e-01]], device='cuda:0',
       grad_fn=<SoftmaxBackward>)

# Soft Voting Evaluation

- pre-training model로 training dataset을 evaluation 하였음 (위의 모델과 동일)
- 위의 개별 모델에 비해 성능이 얼마나 좋아질지 측정하기 위함

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import sentencepiece
from datasets import load_dataset, load_metric
actual_task = 'mnli'
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [ ]:
roberta_nli_model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli').cuda()
roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
roberta_model = [roberta_nli_model, roberta_tokenizer]

distilbart_nli_model = AutoModelForSequenceClassification.from_pretrained('valhalla/distilbart-mnli-12-9').cuda()
distilbart_tokenizer = AutoTokenizer.from_pretrained('valhalla/distilbart-mnli-12-9')
distilbart_model = [distilbart_nli_model, distilbart_tokenizer]

bart_nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').cuda()
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')
bart_model = [bart_nli_model, bart_tokenizer]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def soft_evaluate_model(model, tokenizer_, data) :
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  nli_model =  model
  tokenizer = tokenizer_
  # 0. contradiction   ent  neu
  # 1. neutral  con  ent
  # 2. entailment  neu  con
  true_cnt = 0
  false_cnt = 0

  premise = data['premise']
  hypothesis = data['hypothesis']

  x = tokenizer.encode(premise, hypothesis, return_tensors='pt',truncation_strategy='only_first')
  logits_model = nli_model(x.to(device))[0]

  entail_contradiction_logits = logits_model[:,[2,1,0]]
  probs_model = entail_contradiction_logits.softmax(dim=1)

  return probs_model

In [ ]:
def soft_voting(prob1, prob2, prob3):
  total_probs = torch.div(prob1 + prob2 + prob3, 3)
  return torch.argmax(total_probs).tolist()

In [ ]:
def evaluate_soft_voting(model_1, model_2, model_3, dset):
  roberta_prob = soft_evaluate_model(model_1[0], model_1[1], dset)
  bart_prob = soft_evaluate_model(model_2[0], model_2[1], dset)
  distilbart_prob = soft_evaluate_model(model_3[0], model_3[1], dset)

  voting_result = soft_voting(roberta_prob, bart_prob, distilbart_prob)
  return voting_result # voting_result는 soft-voting 결과로 어느 index의 probability가 가장 높은지 출력함

In [ ]:
# dlen개의 데이터로 soft voting accuracy 측정
dlen = 1000

true_cnt = 0
false_cnt = 0

for i in range(dlen):
  # dlen개의 데이터에서 정확도를 측정
  voting_result = evaluate_soft_voting(roberta_model, bart_model, distilbart_model, dataset['train'][i])
  if voting_result == dataset['train'][i]['label']: true_cnt += 1
  else : false_cnt += 1
  if i % 100 == 0:
    print('평가한 데이터 개수 :', str(i))
    print('soft voting의 출력 :', voting_result)
    print('정답 레이블 :', dataset['train'][i]['label'])
    print('현재 accuracy : {:.2f}%'.format(100*true_cnt/(true_cnt+false_cnt)))
    print('-----------------------')

print('true: {} '.format(true_cnt))
print('flase: {}'.format(false_cnt))
print('soft-voting - accuracy : {:.2f}%'.format(100*true_cnt/(true_cnt+false_cnt)))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2160: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  FutureWarning,


평가한 데이터 개수 : 0
soft voting의 출력 : 1
정답 레이블 : 1
현재 accuracy : : 100.00%
-----------------------
평가한 데이터 개수 : 100
soft voting의 출력 : 2
정답 레이블 : 2
현재 accuracy : : 97.03%
-----------------------
평가한 데이터 개수 : 200
soft voting의 출력 : 2
정답 레이블 : 2
현재 accuracy : : 97.51%
-----------------------
평가한 데이터 개수 : 300
soft voting의 출력 : 2
정답 레이블 : 2
현재 accuracy : : 97.34%
-----------------------
평가한 데이터 개수 : 400
soft voting의 출력 : 1
정답 레이블 : 1
현재 accuracy : : 97.76%
-----------------------
평가한 데이터 개수 : 500
soft voting의 출력 : 1
정답 레이블 : 1
현재 accuracy : : 97.41%
-----------------------
평가한 데이터 개수 : 600
soft voting의 출력 : 0
정답 레이블 : 0
현재 accuracy : : 97.17%
-----------------------
평가한 데이터 개수 : 700
soft voting의 출력 : 2
정답 레이블 : 2
현재 accuracy : : 97.57%
-----------------------
평가한 데이터 개수 : 800
soft voting의 출력 : 2
정답 레이블 : 2
현재 accuracy : : 97.50%
-----------------------
평가한 데이터 개수 : 900
soft voting의 출력 : 0
정답 레이블 : 0
현재 accuracy : : 97.56%
-----------------------
true: 977: 
flase: 23
soft-voting - accuracy : 97.7

In [ ]:
roberta_nli_model = AutoModelForSequenceClassification.from_pretrained('roberta-large-mnli').cuda()
roberta_tokenizer = AutoTokenizer.from_pretrained('roberta-large-mnli')
roberta_model = [roberta_nli_model, roberta_tokenizer]